In [1]:
import tensorflow as tf
import numpy as np
seed = 4
tf.random.set_seed(seed)
np.random.seed(seed)
import os
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input,Activation,GRU
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
from joblib import Parallel,delayed
from scipy.stats import mode
from sklearn.metrics import accuracy_score

/cerebralcortex/kessel_jupyter_virtualenv/tensorflow/lib/python3.7/site-packages/pandas-1.2.3-py3.7-linux-x86_64.egg/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
import pickle
import pandas as pd
def get_participants_df(directory,window_size,min_length):
    df = []
    n = 60//window_size
    for f in os.listdir(directory):
        data = pickle.load(open(directory+f,'rb'))
        df.append([f,data.shape[0]//n])
    df = pd.DataFrame(df,columns=['user','total_test_length'])
    return df[df.total_test_length>=min_length]

def get_training_data(directory,
                      train_length,
                      n_user,
                      participant_df,
                      window_size):
    n = 60//window_size
    users = participant_df['user'].values[:n_user]
    X = []
    y = []
    for f in users:
        df = pickle.load(open(directory+f,'rb'))
        i = np.random.randint(0,df.shape[0]-n*train_length)
        df = df[i:i+n*train_length]
        X.append(np.concatenate(list(df['data'])))
        y.extend([f]*df.shape[0])
    y_dict = {a:i for i,a in enumerate(np.unique(y))}
    y  = [y_dict[a] for a in y]
    return np.concatenate(X),np.array(y),y_dict

def get_trained_model(X_train,y_train,n_timesteps,n_channels,window_size,filepath):
    n_classes = len(np.unique(y_train))
    model =  get_model(input_shape=(n_timesteps,n_channels),n_classes=n_classes)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min',save_weights_only=False)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,patience=40)
    lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=5,verbose=0,factor=0.5)
    callbacks_list = [es,checkpoint]
    train_x,val_x,train_y,val_y = train_test_split(X_train,y_train,test_size=.1,stratify=y_train)
    history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=200, batch_size=300,verbose=0,callbacks=callbacks_list,shuffle=True)
    model.load_weights(filepath)
    print(accuracy_score(val_y,model.predict(val_x).argmax(axis=1)),end=',')
    return model

def get_model(input_shape=(500,3),n_classes=1):
    model =  Sequential()
    model.add(Conv1D(128,2,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Activation('tanh'))
    model.add(Dropout(.5))
    model.add(GRU(128,return_sequences=True,activation='tanh'))
    model.add(Flatten())
    model.add(Dense(400,name='feature',activation='relu'))
    model.add(Dense(n_classes))
    model.add(Dense(n_classes,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='rmsprop')
    return model


window_size = 20
activity = 'walking'
data_directory = './data/'+str(window_size)+'/'+activity+'/'
model_directory = './models/'+str(window_size)+'/'+activity+'/'
min_test_total_length  = 100
fs = 25
n_timesteps,n_channels = fs*window_size,3 
if not os.path.isdir(model_directory):
    os.makedirs(model_directory)
participant_df = get_participants_df(data_directory+'testing/',window_size,min_test_total_length)
n_users = list(np.arange(50,participant_df.shape[0],50))+[participant_df.shape[0]]
train_lengths = np.arange(10,180,10)
n_iters = np.arange(20)

for n_user in n_users[::-1]:
    if not os.path.isdir(model_directory+str(n_user)):
        os.makedirs(model_directory+str(n_user))
    print('--'*30)
    print('Starting for no. of training users = ',n_user)
    for train_length in train_lengths:
        print('Training length minutes = ',train_length)
        if not os.path.isdir(model_directory+str(n_user)+'/'+str(train_length)):
            os.makedirs(model_directory+str(n_user)+'/'+str(train_length))
        for n_iter in n_iters:
            X_train,y_train,user_dict = get_training_data(directory = data_directory+'training/',
                                                          train_length=train_length,
                                                          n_user=n_user,
                                                          participant_df=participant_df,
                                                          window_size=window_size) 
            pickle.dump(user_dict,open(model_directory+str(n_user)+'/'+str(train_length)+'/user_dict_seed_'+str(seed)+'_iteration_'+str(n_iter)+'.p','wb'))
            model = get_model(input_shape=(n_timesteps,n_channels),n_classes=len(np.unique(y_train)))
            model_filepath = model_directory+str(n_user)+'/'+str(train_length)+'/trained_model_seed_'+str(seed)+'_iteration_'+str(n_iter)+'.hdf5'
            model = get_trained_model(X_train,y_train,n_timesteps,n_channels,window_size,model_filepath)
            print('iteration = ',n_iter)
        print(train_length, '--Done')

/cerebralcortex/kessel_jupyter_virtualenv/tensorflow/lib/python3.7/site-packages/pandas-1.2.3-py3.7-linux-x86_64.egg/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


1 2 3
